# (Tabular) MNIST

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import functools

import pred_diff.preddiff as preddiff
from pred_diff.imputers import impute
from pred_diff.imputers import vae_impute as vae
from pred_diff.tools import utils_mnist as ut_mnist
from pred_diff.tools import init_plt

In [ ]:
# paper style
init_plt.update_rcParams(fig_width_pt=234.88*2)

# default
# plt.style.use('default')


Load MNIST data and train a model

In [ ]:
model, df_train, df_test, target_test = ut_mnist.get_model_and_data(max_epochs=1, retrain=False)
print(f"How much overconfident is the model?\n"
      f"temperature rescaling factor: model.T = {model.T:.2f}")

Select parameter for *PredDiff*

In [ ]:
acc = model.return_accurary(data=df_test, target=target_test)
print(f'model accuracy on testset: {acc*100:.1f}%')

In [ ]:
imputer_selection = 'TrainSet'
# imputer_selection = 'TrainSetMahalonobis'
# imputer_selection = 'VAE'

n_group = 10

# possible values: 1, 2, 4, 7, 14
filter_size = 4

In [ ]:
iparam = ut_mnist.ImgParams(n_pixel=28, block_size=filter_size)

if imputer_selection == 'TrainSet':
    explainer = preddiff.PredDiff(model, df_train, imputer_cls=impute.TrainSetImputer, regression=False, n_jobs=8, 
                                  n_group=n_group, fast_evaluation=True)
elif imputer_selection == 'TrainSetMahalonobi':
    explainer = preddiff.PredDiff(model, df_train, imputer_cls=impute.TrainSetMahalanobisImputer, regression=False,
                                  n_jobs=8, n_estimators=20, n_group=n_group, fast_evaluation=True)

elif imputer_selection == 'VAE':
    def df_round_clip(df):
        return df.clip(lower=0, upper=1).div(1. / 255).round().div(255.)
    explainer = preddiff.PredDiff(model, df_train, imputer_cls=vae.VAEImputer, regression=False, gpus=0, epochs=2,
                           custom_postprocessing_fn=df_round_clip, n_group=n_group, fast_evaluation=True)

else:
    assert False, f'please enter a valid imputer_selection = {imputer_selection}'

Select data

In [ ]:
data_selection = 'PaperSelection'
# data_selection = 'RandomSelection'
# data_selection = 'All'

In [ ]:
if data_selection == 'PaperSelection':
    data = df_test.iloc[[4, 15, 84, 9]]         # one digits each: 4, 5, 8, 9
elif data_selection == 'RandomSelection':
    data = df_test.iloc[np.random.randint(low=0, high=df_test.shape[0], size=2)]
elif data_selection == 'All':
    data = df_test.iloc[:]
else:
    assert False, f'please enter a valid data_selection = {data_selection}'

In [ ]:
# calculate relevances
m_relevance, prediction_prob, m_list = ut_mnist.get_relevances(explainer=explainer, data=data, img_params=iparam)

In [ ]:
plot_selection = 'PredictedClass'
# plot_selection = 'FourClasses'
# plot_selection = 'AllClasses'


In [ ]:
for img_id in np.arange(data.shape[0]):
    n_importance = 1
    i_reference = ut_mnist.get_reference_pixel(m_relevance=m_relevance, prediction_prob=prediction_prob,
                                               img_id=img_id, n_importance=n_importance)

    m_interaction = ut_mnist.get_interaction(explainer=explainer, data=data, iparam=iparam, m_list=m_list,
                                             i_reference=i_reference)

    i_vertical, i_horizontal = divmod(i_reference, iparam.max_index)

    rect = functools.partial(ut_mnist.plot_rect, i_reference=i_reference, iparam=iparam)

    if plot_selection == 'PredictedClass':
        ut_mnist.plot_predicted_digit(relevance=m_relevance, interaction=m_interaction, prob_classes=prediction_prob,
                                      data_digit=data, rect=rect, img_params=iparam, image_id=img_id)

    elif plot_selection == 'FourClasses':
        ut_mnist.plot_comparison(m_list_collected=m_relevance, prob_classes=prediction_prob, data_digit=data,
                                 img_params=iparam, image_id=img_id)

    elif plot_selection == 'AllClasses':
        ut_mnist.plot_all_digits(m_list_collected=m_relevance, prob_classes=prediction_prob, data=data,
                                 img_params=iparam, img_id=img_id, imputer='trainset')
        ut_mnist.plot_all_digits(m_list_collected=m_interaction, prob_classes=prediction_prob, data=data,
                                 img_params=iparam, img_id=img_id, imputer='interaction', rect=rect)
